In [1]:
import jieba
import math

In [4]:
s1 = '这只皮靴号码大了。那只号码合适'
s1_cut = [i for i in jieba.cut(s1, cut_all=True) if i != '']
s2 = '这只皮靴号码不小，那只更合适'
s2_cut = [i for i in jieba.cut(s2, cut_all=True) if i != '']
print(s1_cut)
print(s2_cut)
word_set = set(s1_cut).union(set(s2_cut))
print(word_set)

word_dict = dict()
i = 0
for word in word_set:
    word_dict[word] = i
    i += 1
print(word_dict)

s1_cut_code = [word_dict[word] for word in s1_cut]
print(s1_cut_code)
s1_cut_code = [0]*len(word_dict)

for word in s1_cut:
    s1_cut_code[word_dict[word]]+=1
print(s1_cut_code)

s2_cut_code = [word_dict[word] for word in s2_cut]
print(s2_cut_code)
s2_cut_code = [0]*len(word_dict)
for word in s2_cut:
    s2_cut_code[word_dict[word]]+=1
print(s2_cut_code)

# 计算余弦相似度
sum = 0
sq1 = 0
sq2 = 0
for i in range(len(s1_cut_code)):
    sum += s1_cut_code[i] * s2_cut_code[i]
    sq1 += pow(s1_cut_code[i], 2)
    sq2 += pow(s2_cut_code[i], 2)

try:
    result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)), 2)
except ZeroDivisionError:
    result = 0.0
print(result)

['这', '只', '皮靴', '号码', '大', '了', '那', '只', '号码', '合适']
['这', '只', '皮靴', '号码', '不小', '那', '只', '更合', '合适']
{'这', '合适', '只', '大', '了', '那', '号码', '皮靴', '更合', '不小'}
{'这': 0, '合适': 1, '只': 2, '大': 3, '了': 4, '那': 5, '号码': 6, '皮靴': 7, '更合': 8, '不小': 9}
[0, 2, 7, 6, 3, 4, 5, 2, 6, 1]
[1, 1, 2, 1, 1, 1, 2, 1, 0, 0]
[0, 2, 7, 6, 9, 5, 2, 8, 1]
[1, 1, 2, 0, 0, 1, 1, 1, 1, 1]
0.81


# 余弦定理

In [2]:
def simicos(str1, str2):
    # 对两个要计算的字符串进行分词, 使用隐马尔科夫模型(也可不用)
    # 由于不同的分词算法, 所以分出来的结果可能不一样
    # 也会导致相似度会有所误差, 但是一般影响不大
    # 如果想把所有的词性都计算，那么把if及其后面的全部删除掉即可
    cut_str1 = [w for w in jieba.cut(str1, HMM=False)]
    cut_str2 = [w for w in jieba.cut(str2, HMM=False)]
    # 列出所有词
    all_words = set(cut_str1 + cut_str2)
    # 计算词频
    freq_str1 = [cut_str1.count(x) for x in all_words]
    freq_str2 = [cut_str2.count(x) for x in all_words]
    # 计算相似度
    sum_all = sum([x*y for x, y in zip(freq_str1, freq_str2)])
    sqrt_str1 = math.sqrt(sum(x ** 2 for x in freq_str1))
    sqrt_str2 = math.sqrt(sum(x ** 2 for x in freq_str2))
    
    print(sqrt_str1, sqrt_str2)
    return sum_all / (sqrt_str1 * sqrt_str2)

In [23]:
simicos('快速借款逾期有多严重', '逾期问题严重吗')

2.449489742783178 2.0


0.4082482904638631

In [14]:
import jieba.analyse
from math import sqrt
from __future__ import division

class Similarity():
    def __init__(self, target1, target2, topK=10):
        self.target1 = target1
        self.target2 = target2
        self.topK = topK
 
    def vector(self):
        self.vdict1 = {}
        self.vdict2 = {}
        top_keywords1 = jieba.analyse.extract_tags(self.target1, topK=self.topK, withWeight=True)
        top_keywords2 = jieba.analyse.extract_tags(self.target2, topK=self.topK, withWeight=True)
        for k, v in top_keywords1:
            self.vdict1[k] = v
        for k, v in top_keywords2:
            self.vdict2[k] = v
 
    def mix(self):
        for key in self.vdict1:
            self.vdict2[key] = self.vdict2.get(key, 0)
        for key in self.vdict2:
            self.vdict1[key] = self.vdict1.get(key, 0)
 
        def mapminmax(vdict):
            """计算相对词频"""
            _min = min(vdict.values())
            _max = max(vdict.values())
            _mid = _max - _min
            #print _min, _max, _mid
            for key in vdict:
                vdict[key] = (vdict[key] - _min)/_mid
            return vdict
 
        self.vdict1 = mapminmax(self.vdict1)
        self.vdict2 = mapminmax(self.vdict2)
 
    def similar(self):
        self.vector()
        self.mix()
        sum = 0
        for key in self.vdict1:
            sum += self.vdict1[key] * self.vdict2[key]
        A = sqrt(reduce(lambda x,y: x+y, map(lambda x: x*x, self.vdict1.values())))
        B = sqrt(reduce(lambda x,y: x+y, map(lambda x: x*x, self.vdict2.values())))
        return sum/(A*B)

In [15]:
s = Similarity('这只皮靴号码大了。那只号码合适', '这只皮靴号码不小，那只更合适啊', 10)

In [17]:
s.topK

10

# TF计算

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.linalg import norm
 
def tf_similarity(s1, s2):
    def add_space(s):
        return ' '.join(jieba.cut(s))
    
    # 将字中间加入空格
    s1, s2 = add_space(s1), add_space(s2)
    # 转化为TF矩阵
    cv = CountVectorizer(tokenizer=lambda s: s.split())
    corpus = [s1, s2]
    vectors = cv.fit_transform(corpus).toarray()
    # 计算TF系数
    return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))
 

s1 = '这只皮靴号码大了。那只号码合适'
s2 = '这只皮靴号码不小，那只更合适啊'
print(tf_similarity(s1, s2))

0.7161148740394329


# TF-IDF计算

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.linalg import norm
 
def tfidf_similarity(s1, s2):
    def add_space(s):
        return ' '.join(jieba.cut(s))
    
    # 将字中间加入空格
    s1, s2 = add_space(s1), add_space(s2)
    # 转化为TF矩阵
    cv = TfidfVectorizer(tokenizer=lambda s: s.split())
    corpus = [s1, s2]
    vectors = cv.fit_transform(corpus).toarray()
    # 计算TF系数
    return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))
 

s1 = '这只皮靴号码大了。那只号码合适'
s2 = '这只皮靴号码不小，那只更合适'
print(tfidf_similarity(s1, s2))

0.6113454416681379


In [95]:
import json
import codecs
similar_json = json.load(codecs.open('./model/finance.json', 'r', encoding='UTF-8'))

# Word2Vec

In [2]:
import gensim
import jieba
import numpy as np
from scipy.linalg import norm
 
model_file = r'./model/finance.model'
model = gensim.models.Word2Vec.load(model_file)

def get_vec(sentence, model):
    vec = np.zeros(model.vector_size)
    n = 0
    for word in jieba.cut(sentence):
        try:
            vec += model.wv[word]
            n += 1
        except KeyError:
            pass
    return vec if n==0 else vec/n 

def vector_similarity(s1, s2):
    v1, v2 = get_vec(s1, model), get_vec(s2, model)
    return np.dot(v1, v2) / (norm(v1) * norm(v2))

In [3]:
import pandas as pd
df = pd.read_csv('credit_card_qa.csv')
qa_dict = df.to_dict(orient='records')

In [90]:
for qa in qa_dict:
    score = vector_similarity('逾期会有什么影响', qa['question'])
    if score > 0.8:
        print(qa['answer'], score, qa['question'])
        break

用卡爸爸。可以申请账单智能还款 0.818841074409692 信用卡逾期次数多会有什么影响？


# Word2Vec 2

In [4]:
def simq(q1, q2):
    # q1 待匹配问句
    # q2 数据库问句
    q1_words = list(set(jieba.lcut(q1)))
    q2_words = list(set(jieba.lcut(q2)))
    score = 0
    for word in q2_words:
        try:
            if word in q1_words:
                score+=1
            else:
                score+=np.mean([similar_json[word][w] for w in similar_json[word].keys() if w in q1_words and w in similar_json[word]])
        except:
            continue
    return score/len(q2_words)

In [22]:
scores = []
for qa in qa_dict:
    score = simq('逾期会有什么影响', qa['question'])
    if score > 0.5:
        scores.append(score)
#         print(qa['answer'], score, qa['question'])

In [19]:
scores.append(0.6666666666666666)

In [25]:
min(scores)

0.5555555555555556

In [101]:
import jieba.analyse as jieba_analyse

In [102]:
def _():
    tags = jieba_analyse.extract_tags('逾期会有什么影响', topK=len(sentence), withWeight=True)
    words, weights = [], []
    for word, weight in tags:
        words.append(word)
        weights.append(weight)
    sum_of_weights = sum(weights)
    weights = [w/sum_of_weights for w in weights]
    return words, weights

In [103]:
_()

(['逾期', '影响', '什么'],
 [0.5578978511722604, 0.23331826955275922, 0.20878387927498032])

In [104]:
tags = jieba_analyse.extract_tags('逾期会有什么影响', topK=len(sentence), withWeight=True)

In [105]:
tags

[('逾期', 3.1770765998366666), ('影响', 1.3286841183433333), ('什么', 1.1889674353)]